# Part 2: Working with DICOM

This notebook is part of the **Radiotherapy image data analysis using Python** Workshop at ASMIRT 2023 in Sydney, Australia.

In this part you will learn about the DICOM standard and how we can use Python to:
- Read DICOM files
- Inspect DICOM header attributes
- Manipulate DICOM data

## Import libraries and download some sample data

First we will import some libraries that we will need and download some DICOM data which we can use
for these examples.

In [ ]:
import tempfile
import zipfile
import requests
from pathlib import Path

import pydicom

dicom_zip_url = "https://unsw-my.sharepoint.com/:u:/g/personal/z3523015_ad_unsw_edu_au/EfuOALdQEHtFph3EzdpmbOUBx3-kPcLGpuQI2sML7vje-g?download=1"
output_directory = "dicom"

with tempfile.TemporaryDirectory() as temp_dir:
    temp_file = Path(temp_dir).joinpath("tmp.zip")
        
    data = requests.get(dicom_zip_url)
    with open(temp_file, 'wb')as out_file:
        out_file.write(data.content)
        
    with zipfile.ZipFile(temp_file, "r") as zip_ref:
        zip_ref.extractall(output_directory)

## Read DICOM files

Here we will load some of the DICOM files we downloaded. First we will load one slice of a CT
image.

In [ ]:
# Define the path to the folder containing a CT image series
path_to_ct = Path("dicom/HNSCC/HNSCC-01-0019/07-04-1998-NA-RT SIMULATION-48452/5.000000-NA-38976")

# Fetch all DICOM files (with .dcm extension) in that folder
ct_files = list(path_to_ct.glob("*.dcm"))

# Use the pydicom library to read one file from the ct_files list
ct = pydicom.read_file(ct_files[40])

The CT image pixel data is stored in an array we can access using `ct.pixel_array`. We can
use this to visualise the image slice.

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(ct.pixel_array, interpolation='nearest', cmap=plt.get_cmap('gray'))
plt.show()

Similarly we can also read an `RTSTRUCT` and `RTDOSE` DICOM file

In [ ]:
path_to_rtstruct = Path("dicom/HNSCC/HNSCC-01-0019/07-04-1998-NA-RT SIMULATION-48452/1.000000-NA-10361/1-1.dcm")
path_to_rtdose = Path("dicom/HNSCC/HNSCC-01-0019/07-04-1998-NA-RT SIMULATION-48452/1.000000-NA-46284/1-1.dcm")

rtstruct = pydicom.read_file(path_to_rtstruct)
rtdose = pydicom.read_file(path_to_rtdose)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(rtdose.pixel_array[:,:, 40], interpolation='nearest', cmap=plt.get_cmap('gray'))
plt.show()

## Inspect DICOM Header Attributes

Next we can start to explore the DICOM header attributes and the information they contain.

In [ ]:
# Let's take a look at the CT Header Attributes
print(ct)

In [ ]:
# The contour definitions are contained within the DICOM Header attributes of the RTSTRUCT file
# Here we will use some Python code to loop over each contour name we have available

for struct_seq in rtstruct.StructureSetROISequence:
    print(f"{struct_seq.ROINumber}: {struct_seq.ROIName}")

In [ ]:
# Now let's find the contour sequence for a specific structure (Mandible)
mandible_ref_number = 17
mandible_struct = None

for struct_seq in rtstruct.ROIContourSequence:
    if struct_seq.ReferencedROINumber == mandible_ref_number:
        mandible_struct = struct_seq

In [ ]:
print(f"Number of slices on which Mandible is contoured: {len(mandible_struct.ContourSequence)}")
print()
print(mandible_struct.ContourSequence[0])
print()
print(mandible_struct.ContourSequence[0].ContourData)

## Manipulate DICOM Header data

We can use the `pydicom` to modify DICOM Header attributes and then save the modified DICOM files.

In [ ]:
# Let's change the StudyDescription 
print(f"Old Study Description: {rtstruct.StudyDescription}")
rtstruct.StudyDescription = "Corrected Study Description"
print(f"New Study Description: {rtstruct.StudyDescription}")

In [ ]:
# Perhaps a more useful example would be to correct some contour names
# Loop over each contour and change the name Cord to SpinalCord
for struct_seq in rtstruct.StructureSetROISequence:
  if struct_seq.ROIName == "Cord":
    print(f"Renaming {struct_seq.ROIName} to SpinalCord")
    struct_seq.ROIName = "SpinalCord"

In [ ]:
# And finally, we can save the modified RTSTRUCT file
rtstruct.save_as("RS.modified.dcm")

## Exercise

Let's try modifying some more DICOM Header attributes, this time in the RTDOSE file, and save the
modified DICOM file.

In [ ]:
# Modify the StudyDescription of the rtdose DICOM object.



In [ ]:
# Take a look at the other header attributes in the rtdose DICOM. Modify one of the other DICOM
# header attributes.



In [ ]:
# Save the modified rtdose object as a file named RD.modified.dcm

